In [26]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import numpy as np
import scipy.spatial as scipy_spatial

In [2]:
from utils import plot_color_mesh
from utils import select_stars
import lsst.daf.persistence as daf_persistence
from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask
from lsst.meas.algorithms import LoadIndexedReferenceObjectsConfig
import lsst.geom as lsst_geom

In [3]:
hsc_dir = '/project/danielsf/hsc_reprocessing/190930/single_visit'
assert os.path.isdir(hsc_dir)
hsc_butler = daf_persistence.Butler(hsc_dir)

In [4]:
ref_cat_name = 'ps1_pv3_3pi_20170110'

In [5]:
g_visits=[6320, 34338, 34342, 34362, 34366, 34382, 34384, 34400, 34402, 34412, 34414,
          34422, 34424, 34448, 34450, 34464, 34468, 34478, 34480, 34482, 34484, 34486]
r_visits=[7138, 34640, 34644, 34648, 34652, 34664, 34670, 34672, 34674, 34676, 34686,
          34688, 34690, 34698, 34706, 34708, 34712, 34714, 34734, 34758, 34760, 34772]
i_visits=[35870, 35890, 35892, 35906, 35936, 35950, 35974, 36114, 36118, 36140, 36144,
          36148, 36158, 36160, 36170, 36172, 36180, 36182, 36190, 36192, 36202, 36204,
          36212, 36214, 36216, 36218, 36234, 36236, 36238, 36240, 36258, 36260, 36262]
z_visits=[36404, 36408, 36412, 36416, 36424, 36426, 36428, 36430, 36432, 36434, 36438,
          36442, 36444, 36446, 36448, 36456, 36458, 36460, 36466, 36474, 36476, 36480,
          36488, 36490, 36492, 36494, 36498, 36504, 36506, 36508, 38938, 38944, 38950]
y_visits=[34874, 34942, 34944, 34946, 36726, 36730, 36738, 36750, 36754, 36756, 36758,
          36762, 36768, 36772, 36774, 36776, 36778, 36788, 36790, 36792, 36794, 36800,
          36802, 36808, 36810, 36812, 36818, 36820, 36828, 36830, 36834, 36836, 36838]

visit_dict = {}
visit_dict['g'] = g_visits
visit_dict['r'] = r_visits
visit_dict['i'] = i_visits
visit_dict['z'] = z_visits
visit_dict['y'] = y_visits

In [9]:
ccd_num_list = list(range(0,9))+list(range(10,104))

In [46]:
import time
coord_ra = []
coord_dec = []
g_flux = []
calib_phot_used = []
calib_phot_reserved = []
t_start = time.time()
ct=0
for visit_num in visit_dict['g']:
    for ccd_num in ccd_num_list:
        data_id = {'visit': visit_num, 'ccd':ccd_num}
        src = hsc_butler.get('src', dataId=data_id)
        calib = hsc_butler.get('calexp_photoCalib', dataId=data_id)
        valid = (src['calib_photometry_used'] | src['calib_photometry_reserved'])
        src = src[valid]
        g_flux_and_err = calib.instFluxToNanojansky(src, 'base_PsfFlux')
        coord_ra.append(src['coord_ra'])
        coord_dec.append(src['coord_dec'])
        g_flux.append(g_flux_and_err[:,0])
        calib_phot_used.append(src['calib_photometry_used'])
        calib_phot_reserved.append(src['calib_photometry_reserved'])

    ct += 1
    duration = (time.time()-t_start)/3600.0
    per = duration/ct
    pred = per*len(visit_dict['g'])
    print(ct,duration,pred)

coord_ra = np.concatenate(coord_ra)
coord_dec = np.concatenate(coord_dec)
g_flux = np.concatenate(g_flux)
calib_phot_used = np.concatenate(calib_phot_used)
calib_phot_reserved = np.concatenate(calib_phot_reserved)

1 0.0079559635453754 0.17503119799825878
2 0.01476507888899909 0.16241586777899
3 0.02127997789118025 0.1560531712019885
4 0.02776568611462911 0.1527112736304601
5 0.03428844061162737 0.15086913869116042
6 0.04123321056365967 0.15118843873341878
7 0.048182723654641046 0.15143141720030043
8 0.05531454821427663 0.15211500758926072
9 0.06231296731366052 0.15232058676672572
10 0.06893868936432733 0.15166511660152013
11 0.07582401712735494 0.15164803425470988
12 0.08286675479676989 0.15192238379407813
13 0.08981943070888519 0.15200211350734416
14 0.09703338185946146 0.1524810286362966
15 0.10413657036092547 0.15273363652935734
16 0.1112617986732059 0.15298497317565812
17 0.11846763650576274 0.15331105900745767
18 0.12571056829558477 0.15364625013904806
19 0.1330387955904007 0.15404492120993765
20 0.1404508184724384 0.15449590031968224
21 0.14760212865140704 0.15463080144433117
22 0.1550403756565518 0.1550403756565518


In [47]:
print('used ',calib_phot_used.sum())
print('reserved ',calib_phot_reserved.sum())

used  69725
reserved  17438


In [48]:
ref_config = LoadIndexedReferenceObjectsConfig()
ref_config.ref_dataset_name = ref_cat_name
ref_task = LoadIndexedReferenceObjectsTask(hsc_butler, config=ref_config)

In [49]:
ra_mid = np.median(coord_ra)
dec_mid = np.median(coord_dec)
xyz_mid = np.array([np.cos(dec_mid)*np.cos(ra_mid), np.cos(dec_mid)*np.sin(ra_mid), np.sin(dec_mid)])
xyz = np.array([np.cos(coord_dec)*np.cos(coord_ra),
                np.cos(coord_dec)*np.sin(coord_ra),
                np.sin(coord_dec)])

dist = np.arccos(np.dot(xyz.transpose(), xyz_mid))

In [50]:
ref_cat = ref_task.loadSkyCircle(lsst_geom.SpherePoint(ra_mid*lsst_geom.radians, dec_mid*lsst_geom.radians),
                                 lsst_geom.Angle(1.05*dist.max()*lsst_geom.radians),
                                 filterName='g')

In [52]:
del ref_cat_astropy

In [39]:
for line in src.schema:
    name = line[1].getName()
    if 'base_PsfFlux' in name:
        print(line)

(Key<D>(offset=760, nElements=1), Field['D'](name="base_PsfFlux_instFlux", doc="instFlux derived from linear least-squares fit of PSF model", units="count"))
(Key<D>(offset=768, nElements=1), Field['D'](name="base_PsfFlux_instFluxErr", doc="1-sigma instFlux uncertainty", units="count"))
(Key<F>(offset=776, nElements=1), Field['F'](name="base_PsfFlux_area", doc="effective area of PSF", units="pixel"))
(Key['Flag'](offset=640, bit=28), Field['Flag'](name="base_PsfFlux_flag", doc="General Failure Flag"))
(Key['Flag'](offset=640, bit=29), Field['Flag'](name="base_PsfFlux_flag_noGoodPixels", doc="not enough non-rejected pixels in data to attempt the fit"))
(Key['Flag'](offset=640, bit=30), Field['Flag'](name="base_PsfFlux_flag_edge", doc="object was too close to the edge of the image to use the full PSF model"))
(Key<D>(offset=840, nElements=1), Field['D'](name="base_PsfFlux_apCorr", doc="aperture correction applied to base_PsfFlux"))
(Key<D>(offset=848, nElements=1), Field['D'](name="base_

In [45]:
calib.instFluxToNanojansky(src, 'base_PsfFlux')


array([[2.83365425e+03, 4.23136558e+01],
       [4.36405422e+03, 4.49792195e+01],
       [1.36955083e+04, 5.89366853e+01],
       [5.04223434e+04, 1.03302501e+02],
       [1.01155882e+04, 5.51786266e+01],
       [3.05975781e+03, 4.49736843e+01],
       [6.35249088e+03, 4.85085406e+01],
       [1.26032572e+03, 4.19930914e+01],
       [2.94207685e+04, 8.33914278e+01],
       [2.50623403e+03, 4.46072070e+01],
       [6.64680595e+03, 5.53081035e+01],
       [4.28915723e+03, 5.06732063e+01],
       [1.25123847e+04, 6.21246494e+01],
       [8.16810203e+04, 1.47553690e+02],
       [2.77197841e+05, 3.37688066e+02],
       [5.65934145e+03, 4.77046306e+01],
       [5.35069792e+03, 5.11995386e+01],
       [7.18771964e+04, 1.35717199e+02]])

In [44]:
dir(calib)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_getFlux',
 '_getFluxMag0',
 '_getMagnitude',
 '_getThrowOnNegativeFlux',
 '_setThrowOnNegativeFlux',
 'calibrateCatalog',
 'calibrateImage',
 'computeScaledCalibration',
 'computeScalingTo',
 'getCalibrationErr',
 'getCalibrationMean',
 'getFlux',
 'getFluxMag0',
 'getInstFluxAtZeroMagnitude',
 'getLocalCalibration',
 'getMagnitude',
 'getThrowOnNegativeFlux',
 'instFluxToMagnitude',
 'instFluxToNanojansky',
 'isPersistable',
 'magnitudeToInstFlux',
 'readFits',
 'setFluxMag0',
 'setThrowOnNegativeFlux',
 'writeFits']